# 4. Создание и преобразование столбцов с помощью функций

✍ Библиотека Pandas предоставляет большое количество возможностей для преобразований данных, однако иногда необходимо совершать более сложные манипуляции над столбцами. Например, из столбцов, содержащих в себе некоторый текст, необходимо специальным образом извлечь определённые слова, даты или числа.

→ Для таких случаев Pandas не имеет специальных методов, однако позволяет расширить свою функциональность за счёт использования пользовательских функций. 

Мы можем написать некоторую функцию, которая принимает на вход один элемент столбца, каким-то образом его обрабатывает и возвращает результат, после чего применить эту функцию к каждому элементу в столбце с помощью специального метода [apply()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html). В результате применения этой функции будет возвращён объект Series, элементы которого будут представлять результат работы этой функции.

Рассмотрим пример. В наших данных есть столбец с адресами объектов недвижимости. Проблема этого столбца в том, что в нём слишком большое количество уникальных значений: почти на каждый объект недвижимости в таблице приходится свой уникальный адрес. Убедимся в этом, вычислив количество уникальных значений в столбце с помощью метода [nunique()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.nunique.html):

In [1]:
import pandas as pd
melb_data = pd.read_csv('data/melb_data_ps.csv', sep=',')

melb_df = melb_data.copy()

In [2]:
melb_df['Address'].nunique()

13378

Если мы прогнозируем цену объекта, то такое большое количество возможных категорий может плохо сказаться на модели, которую мы бы хотели в дальнейшем построить на наших данных. Говорят, что такой признак, скорее всего, не имеет статистической значимости, потому что не позволяет разделить данные на группы, которые можно сравнить по целевому признаку.

Из-за таких признаков зависимость между целевым признаком, который мы хотим предсказать, и признаками, на основе которых мы делаем предсказание, становится очень сложной. При этом точность моделирования при учёте такого признака может не повыситься, а даже снизиться, а производительность однозначно резко упадёт.

Обычно подобные признаки удаляют, однако можно поступить умнее: давайте извлечём из признака адреса характеристику подтипа улицы (улица, шоссе, авеню, бульвар). Для этого сначала внимательнее посмотрим на структуру адреса, выберем несколько строк столбца Address:

In [3]:
print(melb_df['Address'].loc[177])
print(melb_df['Address'].loc[1812])
print(melb_df['Address'].loc[9001])
# 2/119 Railway St N
# 9/400 Dandenong Rd
# 172 Danks St

2/119 Railway St N
9/400 Dandenong Rd
172 Danks St


Итак, адрес строится следующим образом: сначала указывается номер дома и корпус, после указывается название улицы, а в конце — подтип улицы, но в некоторых случаях к подтипу добавляется географическая отметка (N — север, S — юг и т. д.), она нам не нужна . Для того чтобы выделить подтип улицы, на которой находится объект, можно использовать следующую функцию:

In [4]:
# На вход данной функции поступает строка с адресом.
def get_street_type(address):
# Создаём список географических пометок exclude_list.
    exclude_list = ['N', 'S', 'W', 'E']
# Метод split() разбивает строку на слова по пробелу.
# В результате получаем список слов в строке и заносим его в переменную address_list.
    address_list = address.split(' ')
# Обрезаем список, оставляя в нём только последний элемент,
# потенциальный подтип улицы, и заносим в переменную street_type.
    street_type = address_list[-1]
# Делаем проверку на то, что полученный подтип является географической пометкой.
# Для этого проверяем его на наличие в списке exclude_list.
    if street_type in exclude_list:
# Если переменная street_type является географической пометкой,
# переопределяем её на второй элемент с конца списка address_list.
        street_type = address_list[-2]
# Возвращаем переменную street_type, в которой хранится подтип улицы.
    return street_type


## ДОПОЛНИТЕЛЬНО

Подробнее о методе [split()](https://docs-python.ru/tutorial/operatsii-tekstovymi-strokami-str-python/metod-str-split/)

In [5]:
melb_df['Address'].apply(get_street_type)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

Теперь применим эту функцию к столбцу c адресом. Для этого передадим функцию get_street_type в аргумент метода столбца apply(). В результате получим объект Series, который положим в переменную street_types:



In [6]:
street_types = melb_df['Address'].apply(get_street_type)
display(street_types)

0        St
1        St
2        St
3        La
4        St
         ..
13575    Cr
13576    Dr
13577    St
13578    St
13579    St
Name: Address, Length: 13580, dtype: object

Обратите внимание, что функция пишется для одного элемента столбца, а метод apply() применяется к каждому его элементу. Используемая функция обязательно должна иметь возвращаемое значение.

Итак, мы смогли выделить подтип улицы. Посмотрим, сколько уникальных значений у нас получилось:

In [7]:
print(street_types.nunique())
# 56

56


У нас есть 56 уникальных значений. Однако наш результат можно улучшить. Давайте для начала посмотрим на частоту каждого подтипа улицы с помощью метода value_counts:

In [8]:
display(street_types.value_counts())

St           8012
Rd           2825
Ct            612
Dr            447
Av            321
Gr            311
Pde           211
Pl            169
Cr            152
Cl            100
La             67
Bvd            53
Tce            47
Wy             40
Avenue         40
Cct            25
Hwy            24
Parade         15
Boulevard      13
Sq             11
Crescent        9
Cir             7
Strand          7
Esplanade       6
Grove           5
Gdns            4
Grn             4
Fairway         4
Mews            4
Crossway        3
Righi           3
Victoria        2
Ridge           2
Crofts          2
Esp             2
Glade           1
Gra             1
Ave             1
Woodland        1
Outlook         1
Hts             1
Highway         1
Athol           1
Summit          1
Grand           1
Res             1
Nook            1
Eyrie           1
Dell            1
East            1
Loop            1
Grange          1
Terrace         1
Cove            1
Qy              1
Corso     

In [9]:
street_types.unique()

array(['St', 'La', 'Rd', 'Gr', 'Ct', 'Dr', 'Pde', 'Pl', 'Hwy', 'Parade',
       'Tce', 'Bvd', 'Boulevard', 'Highway', 'Cl', 'Athol', 'Grove',
       'Cct', 'Avenue', 'Righi', 'Esp', 'Crescent', 'Cir', 'Crossway',
       'Grange', 'Sq', 'Res', 'Esplanade', 'Strand', 'Cove', 'Mews',
       'Crofts', 'Qy', 'Glade', 'Nook', 'Gdns', 'Victoria', 'Fairway',
       'Terrace', 'Ridge', 'Loop', 'East', 'Dell', 'Eyrie', 'Grn', 'Gra',
       'Grand', 'Summit', 'Cr', 'Av', 'Wy', 'Hts', 'Outlook', 'Woodland',
       'Ave', 'Corso'], dtype=object)

Из данного вывода можно увидеть, что есть группа наиболее популярных подтипов улиц, а дальше частота подтипов быстро падает.

В таком случае давайте применим очень распространённый метод уменьшения количества уникальных категорий — выделим n подтипов, которые встречаются чаще всего, а остальные обозначим как 'other' (другие).

Для этого к результату метода value_counts применим метод [nlargest()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.nlargest.html), который возвращает n наибольших значений из Series. Зададим n=10, т. е. мы хотим отобрать десять наиболее популярных подтипов. Извлечём их названия с помощью атрибута index, а результат занесём в переменную popular_stypes:

In [10]:
popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)
# Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')


Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')


Попробуйте применить использованные методы последовательно и выводить результат, чтобы проследить за логикой преобразования.

Теперь, когда у нас есть список наиболее популярных подтипов улиц, введём lambda-функцию, которая будет проверять, есть ли строка x в этом перечне, и, если это так, lambda-функция будет возвращать x, в противном случае она будет возвращать строку 'other'. Наконец, применим такую функцию к Series street_types, полученной ранее, а результат определим в новый столбец таблицы StreetType:

In [11]:
melb_df['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df['StreetType']

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

Посмотрим на результирующее число уникальных подтипов:

In [12]:
melb_df['StreetType'].nunique()

11

Теперь, у нас нет потребности хранить признак Address, так как, если конкретное местоположение объекта всё же и влияет на его стоимость, то оно определяется столбцами Longitude и Lattitude. Удалим его из нашей таблицы:

In [13]:
melb_df.drop(['Address', 'index'], axis=1)

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates,StreetType
0,Abbotsford,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067,2,...,202.0,126.0,1970,Yarra,-37.79960,144.99840,Northern Metropolitan,4019,"-37.7996, 144.9984",St
1,Abbotsford,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067,2,...,156.0,79.0,1900,Yarra,-37.80790,144.99340,Northern Metropolitan,4019,"-37.8079, 144.9934",St
2,Abbotsford,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067,3,...,134.0,150.0,1900,Yarra,-37.80930,144.99440,Northern Metropolitan,4019,"-37.8093, 144.9944",St
3,Abbotsford,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067,3,...,94.0,126.0,1970,Yarra,-37.79690,144.99690,Northern Metropolitan,4019,"-37.7969, 144.9969",other
4,Abbotsford,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067,3,...,120.0,142.0,2014,Yarra,-37.80720,144.99410,Northern Metropolitan,4019,"-37.8072, 144.9941",St
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13575,Wheelers Hill,4,h,1245000.0,S,Barry,26/08/2017,16.7,3150,4,...,652.0,126.0,1981,NaN,-37.90562,145.16761,South-Eastern Metropolitan,7392,"-37.90562, 145.16761",Cr
13576,Williamstown,3,h,1031000.0,SP,Williams,26/08/2017,6.8,3016,3,...,333.0,133.0,1995,NaN,-37.85927,144.87904,Western Metropolitan,6380,"-37.85927, 144.87904",Dr
13577,Williamstown,3,h,1170000.0,S,Raine,26/08/2017,6.8,3016,3,...,436.0,126.0,1997,NaN,-37.85274,144.88738,Western Metropolitan,6380,"-37.85274, 144.88738",St
13578,Williamstown,4,h,2500000.0,PI,Sweeney,26/08/2017,6.8,3016,4,...,866.0,157.0,1920,NaN,-37.85908,144.89299,Western Metropolitan,6380,"-37.85908, 144.89299",St


Таким образом, с помощью написания собственных функций и их комбинирования с методом apply() из библиотеки Pandas мы смогли извлечь информацию из признака с адресом и заменить на признак подтипа улицы.

Примечание. Внимательный читатель наверняка обратит внимание на то, что мы допустили небольшую ошибку!

Если присмотреться, то в списке подтипов улиц street_types можно заметить подтипы, которые именуются различным образом, но при этом обозначают одинаковые вещи. Например, подтипы Av и Avenue, Bvd и Boulevard, Pde и Parade. Мы упустили данный момент, хотя в реальных задачах стоит обращать пристальное внимание на результаты преобразований и исправлять неточности в данных.

Такие ошибки в данных (обозначение идентичных категорий различными именами) являются одним из видов «грязных» данных.

Порой отследить такие неточности бывает очень сложно, а при наличии большого количества категорий (например, более ста) — практически невозможно.

Мы предлагаем вам самостоятельно разобраться с этой ошибкой: попробуйте написать функцию-преобразование (lambda-функцию-преобразование), которая возвращала бы вместо значений Avenue, Boulevard и Parade их топографические сокращения, и примените её к данным о подтипах улиц.

→ Обратите внимание, что данное преобразование необходимо применить до сокращения количества уникальных категорий.

In [14]:
def fix_long_street_types(street_type):
    if street_type == "Avenue":
        return "Av"

    if street_type == "Boulevard":
        return "Bvd"

    if street_type == "Parade":
        return "Pde"

    return street_type

In [15]:
street_types.apply(fix_long_street_types).nunique()

53

In [16]:
popular_stypes =street_types.value_counts().nlargest(10).index
print(popular_stypes)

Index(['St', 'Rd', 'Ct', 'Dr', 'Av', 'Gr', 'Pde', 'Pl', 'Cr', 'Cl'], dtype='object')


In [17]:
melb_df_fixed = melb_data.copy()

In [18]:
melb_df_fixed['StreetType'] = street_types.apply(lambda x: x if x in popular_stypes else 'other')
melb_df_fixed['StreetType']

0           St
1           St
2           St
3        other
4           St
         ...  
13575       Cr
13576       Dr
13577       St
13578       St
13579       St
Name: StreetType, Length: 13580, dtype: object

In [19]:
melb_df_fixed['StreetType'].nunique()

11

In [20]:
melb_df_fixed = melb_df_fixed.drop(["Address", "index"], axis=1)

In [21]:
melb_df_fixed.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount,Coordinates,StreetType
0,Abbotsford,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067,2,...,202.0,126.0,1970,Yarra,-37.7996,144.9984,Northern Metropolitan,4019,"-37.7996, 144.9984",St
1,Abbotsford,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067,2,...,156.0,79.0,1900,Yarra,-37.8079,144.9934,Northern Metropolitan,4019,"-37.8079, 144.9934",St
2,Abbotsford,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067,3,...,134.0,150.0,1900,Yarra,-37.8093,144.9944,Northern Metropolitan,4019,"-37.8093, 144.9944",St
3,Abbotsford,3,h,850000.0,PI,Biggin,4/03/2017,2.5,3067,3,...,94.0,126.0,1970,Yarra,-37.7969,144.9969,Northern Metropolitan,4019,"-37.7969, 144.9969",other
4,Abbotsford,4,h,1600000.0,VB,Nelson,4/06/2016,2.5,3067,3,...,120.0,142.0,2014,Yarra,-37.8072,144.9941,Northern Metropolitan,4019,"-37.8072, 144.9941",St


Резюмируя, поделимся общими рекомендациями по уменьшению числа уникальных значений в признаке, который описывается категориями:

1
Определите (хотя бы на глаз) соотношение числа уникальных категорий интересующего вас признака к общему числу объектов в таблице. Если это соотношение превышает значение 30 %, то это уже повод задуматься над уменьшением числа категорий и перейти к шагу 2.

2
Если ваш признак уникален для каждого объекта, например адрес, имя или название, то такой признак, скорее всего, не имеет статистической значимости. От таких признаков чаще всего избавляются. Однако можно попробовать выделить из этого признака какие-то общие черты, например, как мы это сделали с подтипами улиц. Такой же трюк можно произвести, например, с названиями компаний, в которых может быть скрыт признак типа организации (из строки «ООО Три Слепые Мыши» можно извлечь ООО — общество с ограниченной ответственностью).

Далее переходите к шагу 3.

3
Если даже после преобразования число уникальных категорий всё ещё велико, можно попробовать с помощью метода value_counts() оценить, есть ли в данных категории, которые употребляются гораздо реже, чем остальные. Если такие категории присутствуют, переходите к шагу 4.

4
Можно подобрать число  популярных категорий таким образом, чтобы эти категории покрывали большую часть ваших данных.

Когда вы выбрали оптимальное число, переходите к шагу 5.

5
Наконец, можно совершить преобразование, обозначив категории, не попавшие в число популярных, как «другие».

→ Такая методика является очень популярной и хорошо показывает себя на практике. Однако не нужно ей злоупотреблять: применяя эту методику ко всем столбцам подряд, вы рискуете потерять немалую долю полезной информации из ваших данных. Внимательно изучите интересующий вас признак, прежде чем преобразовывать его.

✍ А теперь предлагаем вам закрепить пройденный материал и потренироваться использовать преобразования с помощью функций ↓

### Задание 4.1
Загляните в [документацию по методу apply()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) и определите, с помощью какого параметра данного метода можно передавать другие аргументы в вызываемую функцию.
- new_args
- args
- axis
- result_type

Ответ: args

### Задание 4.2
Ранее, в задании 3.3, мы создали признак WeekdaySale в таблице melb_df — день недели продажи. Из полученных в задании результатов можно сделать вывод, что объекты недвижимости в Мельбурне продаются преимущественно по выходным (суббота и воскресенье).
Напишите функцию get_weekend(weekday), которая принимает на вход элемент столбца WeekdaySale и возвращает 1, если день является выходным, и 0 — в противном случае, и создайте столбец Weekend в таблице melb_df с помощью неё.

Примените эту функцию к столбцу и вычислите среднюю цену объекта недвижимости, проданного в выходные дни. Результат округлите до целых.



In [22]:
melb_df_fixed["Date"] = pd.to_datetime(melb_df_fixed["Date"], dayfirst="True")
melb_df_fixed["WeekDaySale"] = melb_df_fixed["Date"].dt.weekday

In [23]:
def get_weekend(weekday):
    return weekday in [5, 6]

In [24]:
melb_df_fixed['WeekDaySale'].apply(get_weekend)

0         True
1        False
2         True
3         True
4         True
         ...  
13575     True
13576     True
13577     True
13578     True
13579     True
Name: WeekDaySale, Length: 13580, dtype: bool

In [25]:
answer = round(melb_df_fixed[melb_df_fixed['WeekDaySale'].apply(get_weekend)]['Price'].mean())
print(f'Ответ: {answer}')

Ответ: 1081199


### Задание 4.3
Преобразуйте столбец SellerG с наименованиями риелторских компаний в таблице melb_df следующим образом: оставьте в столбце только 49 самых популярных компаний, а остальные обозначьте как 'other'.
Найдите, во сколько раз минимальная цена объектов недвижимости, проданных компанией 'Nelson', больше минимальной цены объектов, проданных компаниями, обозначенными как 'other'. Ответ округлите до десятых.

In [26]:
# Выбираем топ 49 риэлторских компаний по количеству продаж
top_sellers = melb_df_fixed["SellerG"].value_counts().nlargest(49)

# Задаем функцию возвращающую название компании без изменений для
# компаний в списке top_sellers. Для остальных компаний возвращаем
# 'other'
def get_seller_name(seller):
    return seller if seller in top_sellers else "other"


# Изменяем столбец 'SellerG' применяя функцию get_seller_name к названиям компаний
melb_df_fixed["SellerG"] = melb_df_fixed["SellerG"].apply(get_seller_name)

# Определяем минимальную стоимость недвижимости, проданной компанией 'Nelson'
nelson_sales = melb_df_fixed[melb_df_fixed["SellerG"] == "Nelson"]["Price"].min()

# Определяем минимальную стоимость недвижимости, из проданных компаниями в категории 'other'
other_sales = melb_df_fixed[melb_df_fixed["SellerG"] == "other"]["Price"].min()

#  Считаем соотношение
answer = nelson_sales / other_sales

print(f"Ответ: {answer:.1f}")

Ответ: 1.3


### ЗАДАНИЕ 4.4 (ВЫПОЛНЯЕТСЯ В CODEBOARD НИЖЕ)

Представьте, что вы занимаетесь подготовкой данных о вакансиях с платформы hh.ru. В вашем распоряжении имеется таблица, в которой с помощью парсинга собраны резюме кандидатов. В этой таблице есть текстовый столбец «Опыт работы». Пример такого столбца представлен ниже в виде объекта Series. Структура текста в столбце фиксирована и не может измениться.

Напишите функцию get_experience(arg), аргументом которой является строка столбца с опытом работы. Функция должна возвращать опыт работы в месяцах. Не забудьте привести результат к целому числу.

Обратите внимание, что опыт работы может выражаться только в годах или только в месяцах. Учтите это при построении своей функции. Кроме того, учтите возможные вариации слов месяц (месяца, месяцев) и год (года, лет).

Примените вашу функцию к Series experience_col с помощью метода apply().

Пример результата работы функции get_experience:

![исходные данные](./img/dst3-u1-md11_4_4.png) → ![данные после обработки](./img/dst3-u1-md11_4_5.png)

In [42]:
def get_experience(arg):
    # Задаем списки токенов для года и месяца
    years_tokens = ["год", "года", "лет"]
    months_tokens = ["месяц", "месяца", "месяцев"]

    # Формируем список слов после "опыт работы "
    words = arg[len("опыт работы ") :].lower().split(" ")

    # Если первое слово в списке можно преобразовать в целое
    # и следующее за ним слово входит в список years_tokens
    if words[0].isdigit() and words[1] in years_tokens:
        # Преобразуем первое слово в число лет
        years = int(words[0])
        # Удаляем из списка два первых слова
        words = words[2:]
    else:
        # Иначе число лет равно нулю
        years = 0

    # Если осталось меньше 2 слов
    if len(words) < 2:
        # Число месяцев равно нулю
        months = 0
    # Иначе если первое слово в списке можно преобразовать в число
    # и следующее за ним слово входит в список months_tokens
    elif words[0].isdigit() and words[1] in months_tokens:
        # Преобразуем первое слово в число лет
        months = int(words[0])
    else:
        # Иначе число месяцев равно нулю
        months = 0

    # Возвращаем общее число месяцев
    return years * 12 + months

In [41]:
# Тесты
assert(get_experience('Опыт работы 8 лет 3 месяца') == 99)
assert(get_experience('Опыт работы 3 года 5 месяцев') == 41)
assert(get_experience('Опыт работы 1 год 9 месяцев') == 21)
assert(get_experience('Опыт работы 3 месяца') == 3)
assert(get_experience('Опыт работы 6 лет') == 72)

### Сохраним данные для следующего модуля

In [44]:
melb_df_fixed.to_csv('data/melb_data_4.csv', sep = ',', index=False)